# K-Mer based workflow for scRNA-seq

* Compile needed annotation files
* Build needed references
* Quantify against reference

## Get and massage annotation files needed for Salmon human

Using all Ensembl-99 as ground truth

## Make human genome annotation directory

In [ ]:
%%bash

mkdir /input_dir/corona_analysis/annotations/human
cd /input_dir/corona_analysis/annotations/human


## Get human genome fasta
Get human fasta file using 2bit from UCSC


In [ ]:
%%bash
curl -s -L ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.gz > \
    /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.gz
gunzip Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.gz
#Toss everything except canonical chromosomes
awk "/^>/ {n++} n>25 {exit} {print}" /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa > GRCh38_filt_dna_sm.fa


## Get human gene annotations

Get human gene annotations in GTF form from Gencode (v33)


In [ ]:
%%bash

cd /input_dir/corona_analysis/annotations/human/
#Get ensembl gtf file
curl -s -L ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz > \
    /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.99.gtf.gz
gunzip /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.99.gtf.gz


# Make genome annotations for STAR and RSEM


In [ ]:
%%bash
# Make concatenated GTF of hg38 Ensembl, retroposon, and CoVid
cd /data_dir/corona_analysis/annotations/human/
cat Homo_sapiens.GRCh38.99.gtf \
    retroposon_hg38.gtf \
    /data_dir/corona_analysis/annotations/CoVid/jx_s2_covid.gtf > \
    Homo_sapiens.GRCh38.99_retroposon_covid.gtf

In [ ]:
%%bash
#Make covid + human softmasked genome
cd /input_dir/corona_analysis/annotations/human/
cat GRCh38_filt_dna_sm.fa /data_dir/corona_analysis/annotations/CoVid/EPI_ISL_407193_edit.fasta > \
    GRCh38_filt_dna_sm_covid.fa


In [ ]:
%%bash
#Generate genome indices for STAR
mkdir /input_dir/corona_analysis/annotations/human/STAR_ix
STAR --runThreadN 12 \
     --runMode genomeGenerate --outTmpDir /input_dir/corona_analysis/temp/star2 \
     --genomeDir /input_dir/corona_analysis/annotations/human/STAR_ix \
     --genomeFastaFiles /input_dir/corona_analysis/annotations/human/GRCh38_filt_dna_sm_covid.fa \
     --sjdbGTFfile /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.99_retroposon_covid.gtf \
     --sjdbOverhang 149 --limitGenomeGenerateRAM 30000000000 --genomeSAsparseD 2


In [ ]:
%%bash

#Generate genome annotations for RSEM
rsem-prepare-reference --gtf /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.99.gtf  \
                       /input_dir/corona_analysis/annotations/human/GRCh38_filt_dna_sm.fa \
                       /input_dir/corona_analysis/annotations/human/RSEM_ix 


## Get hard masked hg38 human genome

In [ ]:
%%bash

cd /input_dir/corona_analysis/annotations/human/

#Get hard masked hg38 genome and filter for only canonical chromosomes
curl -s -L ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna_rm.primary_assembly.fa.gz > \
    /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.dna_rm.primary_assembly.fa.gz
gunzip Homo_sapiens.GRCh38.dna_rm.primary_assembly.fa.gz
#Toss everything except canonical chromosomes
awk "/^>/ {n++} n>25 {exit} {print}" /input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.dna_rm.primary_assembly.fa > GRCh38_filt_dna_rm.fa
#Get list of decoy sequences for salmon
grep "^>" /input_dir/corona_analysis/annotations/human/GRCh38_filt_dna_rm.fa | cut -d " " -f 1 > decoys.txt
sed -i.bak -e 's/>//g' decoys.txt


## Begin making sequence cDNA files for salmon

In [ ]:
%%bash

#Get cDNA of transcripts for hg38
wget ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
gunzip Homo_sapiens.GRCh38.cdna.all.fa.gz

#Filter the transcripts for only those in STAR 
cut -f 1 /input_dir/corona_analysis/annotations/human/STAR_ix/transcriptInfo.tab > in_enst_names.txt
awk '{ if ((NR>1)&&($0~/^>/)) { printf("\n%s", $0); } else if (NR==1) { printf("%s", $0); } else { printf("\t%s", $0); } }' \
    Homo_sapiens.GRCh38.cdna.all.fa \
    | grep -Ff in_enst_names.txt - \
    | tr "\t" "\n" > Hg38_cdna_filt.fa

# coding_genes_hgnc.txt = list of all HGNC protein coding symbols
#Filter the transcripts to ONLY protein coding
awk '{ if ((NR>1)&&($0~/^>/)) { printf("\n%s", $0); } else if (NR==1) { printf("%s", $0); } else { printf("\t%s", $0); } }' \
    Hg38_cdna_filt.fa \
    | grep -Ff coding_genes_hgnc.txt - \
    | tr "\t" "\n" > Hg38_cdna_coding_filt.fa


## scRNA-seq chromium barcodes

In [2]:
%%bash
#All available here: https://github.com/10XGenomics/cellranger/tree/master/lib/python/cellranger/barcodes
##Get whitelist for version 10x v1
#curl -s -L https://raw.githubusercontent.com/10XGenomics/cellranger/master/lib/python/cellranger/barcodes/737K-april-2014_rc.txt > \
#    /input_dir/corona_analysis/annotations/human/scRNA_10x_v1_whitelist.txt
##Get whitelist for version 10x v2
#curl -s -L https://raw.githubusercontent.com/10XGenomics/cellranger/master/lib/python/cellranger/barcodes/737K-august-2016.txt > \
#    /input_dir/corona_analysis/annotations/human/scRNA_10x_v2_whitelist.txt
#Get whitelist for version 10x v3
curl -s -L  wget https://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/3M-february-2018.txt.gz > \
    /input_dir/corona_analysis/annotations/human/scRNA_10x_v3_whitelist.txt.gz
gunzip /input_dir/corona_analysis/annotations/human/scRNA_10x_v3_whitelist.txt.gz


### Get rRNA from 

https://www.genenames.org/data/genegroup/#!/group/848

Flip to ENST using Biomart and feed into rRNA_ensembl.txt to get rRNA_ensembl_cdna.fa



## Get LINE element cDNA and make GTF

In [ ]:
%%bash 
#Add custom relevant T-cell loci
cd /data_dir/corona_analysis/annotations/human/
#Add LINE element loci
sed 's/^chr//' /data_dir/corona_analysis/annotations/hsflnil1_hg38_sorted.bed  > t.bed
#Add innate immunity loci
echo -ne "19\t47903455\t47963540\tLoci_hCRISPR1\t0\t+\t0\t0\t128,128,128\n" >> t.bed
echo -ne "19\t50090483\t50148547\tLoci_hCRISPR2\t0\t+\t0\t0\t128,128,128\n" >> t.bed
echo -ne "19\t49020448\t49057114\tLoci_hCRISPR3\t0\t+\t0\t0\t128,128,128\n" >> t.bed
echo -ne "7\t142440833\t142539173\tLoci_hCRISPR1_TCRB\t0\t+\t0\t0\t128,128,128\n" >> t.bed
echo -ne "7\t38340582\t38379445\tLoci_hCRISPR2_TCRG\t0\t+\t0\t0\t128,128,128\n" >> t.bed
    
sort-bed t.bed > t2.bed
bedToGenePred t2.bed t2.genepred
genePredToGtf file ./t2.genepred retroposon_hg38.gtf

bedtools getfasta -name -fi GRCh38_filt_dna_sm.fa -bed t2.bed -fo hsflnil1_hg38_sorted.fa
rm t.bed  t2.bed t2.genepred t2.gtf


## Get CoVid cDNA


In [ ]:
%%bash

#Add cDNA of CoVid 
cd /data_dir/corona_analysis/annotations/CoVid
bedtools getfasta -fi EPI_ISL_407193_edit.fasta -bed jx_s2_covid.gtf > EPI_ISL_407193_edit_cdna.fa
#Uniquify transcripts
cat EPI_ISL_407193_edit_cdna.fa | grep '^>' | sed 's/>//' | sort -u > uniq_cdna_covid.txt
seqkit grep -f uniq_cdna_covid.txt EPI_ISL_407193_edit_cdna.fa | seqkit rmdup -n -o EPI_ISL_407193_edit_cdna_uniq.fa


## Concatenate all files together for gentrome

In [ ]:
%%bash
#Make gentrome for Salmon
mkdir /data_dir/corona_analysis/annotations/human/salmon_ann
cd /data_dir/corona_analysis/annotations/human/salmon_ann
cat /data_dir/corona_analysis/annotations/human/Hg38_cdna_coding_filt.fa \
    /data_dir/corona_analysis/annotations/human/rRNA_ensembl_cdna.fa \
    /data_dir/corona_analysis/annotations/human/hsflnil1_hg38_sorted.fa \
    /data_dir/corona_analysis/annotations/CoVid/EPI_ISL_407193_edit_cdna_uniq.fa \
    /data_dir/corona_analysis/annotations/human/GRCh38_filt_dna_rm.fa \
    > gentrome_hg38_filt.fa 

#Full cdna file: Homo_sapiens.GRCh38.cdna.all.fa
#Filtered cdna file: Hg38_cdna_coding_filt.fa


In [ ]:
%%bash
#Reduce ram used w/ sparse
salmon index --sparse -t gentrome_hg38_filt.fa -d ../decoys.txt -p 13 -i salmon_hg38_index


## Make transcript 2 gene map and rRNA / mtRNA annotation

In [ ]:
%%bash

#Get transcript IDs of rRNA w/ version ID
grep ">" /input_dir/corona_analysis/annotations/human/rRNA_ensembl_cdna.fa | sed 's/>//' | \
    awk '{print $1"\trRNA-all"}' > /input_dir/corona_analysis/annotations/human/rRNA_ensembl.txt


In [ ]:
gtf_parse = "/input_dir/corona_analysis/annotations/human/Homo_sapiens.GRCh38.99.gtf"
mt_tran_out = "/input_dir/corona_analysis/annotations/human/gencode_mt.txt"
out_tran_map = "/input_dir/corona_analysis/annotations/human/salmon_grch38_gencode_tran2gene.txt"
rrna_tran_in = "/input_dir/corona_analysis/annotations/human/rRNA_ensembl.txt"

#Keep track of relevant transcripts
rRNA_tran = dict()
tran_gene_dict = dict()
tran_name_dict = dict()

with open(rrna_tran_in, "r") as rRNA_text:
    for line in rRNA_text:
        arr = line.strip().split()
        rRNA_tran[arr[0]] = arr[1]

with open(gtf_parse, "r") as gene_in:
    with open(mt_tran_out,"w") as mito_out:
        for line in gene_in:
            if line.startswith("#"):
                continue
            else:
                arr = line.strip().split("\t")
                if (arr[2] == "transcript"):
                    tmp_arr = arr[-1].strip(';').strip().split(";")
                    tmp_arr = tmp_arr[:-1]
                    arr_tran = [feature.strip().split(" ") for feature in tmp_arr]
                    cur_tran_dict = {key: value for (key, value) in arr_tran}
                    tran_id = cur_tran_dict['transcript_id'].replace("\"","")
                    tran_ver = cur_tran_dict['transcript_version'].replace("\"","")
                    tran_str = f"{tran_id}.{tran_ver}"
                    gene_id = cur_tran_dict['gene_id'].replace("\"","")
                    gene_ver = cur_tran_dict['gene_version'].replace("\"","")
                    gene_name = cur_tran_dict['gene_name'].replace("\"","")
                    #Only add transcript if NOT rRNA
                    if (tran_str not in rRNA_tran):
                        tran_gene_dict[tran_str] = f"{gene_id}.{gene_ver}"
                        tran_name_dict[tran_str] = gene_name
                    #If mitochondrial gene add to mt gene txt file
                    if arr[0] == "MT":
                        mito_out.write(f"{tran_str}\t{gene_name}\n")
                        
with open(out_tran_map, "w") as kal_tran:
    for tran in tran_gene_dict.keys():
        out_line = f"{tran}\t{tran_name_dict[tran]}\n"
        kal_tran.write(out_line)
        

In [ ]:
%%bash

#Add rRNA to transcript2gene map
grep ">" /input_dir/corona_analysis/annotations/human/rRNA_ensembl_cdna.fa | sed 's/>//' | \
    awk '{print $1"\trRNA-all"}' >> /input_dir/corona_analysis/annotations/human/salmon_grch38_gencode_tran2gene.txt


In [ ]:
%%bash

#Add LINE elements to transcript2gene map
grep ">" /input_dir/corona_analysis/annotations/human/hsflnil1_hg38_sorted.fa | \
     sed 's/>//' | awk '{print $1"\t"$1 }' >> /input_dir/corona_analysis/annotations/human/salmon_grch38_gencode_tran2gene.txt


In [ ]:
%%bash

#Add CoVid to transcript2gene map
cd /input_dir/corona_analysis/annotations/CoVid/
grep ">" EPI_ISL_407193_edit_cdna_uniq.fa | \
     sed 's/>//' | awk '{print $1"\tCoVid-19-EPI_ISL_407193" }' >> /input_dir/corona_analysis/annotations/human/salmon_grch38_gencode_tran2gene.txt
